In [ ]:
import re
import os
import csv
import time
import shutil
import PyPDF2
import pandas as pd
from docx import Document
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
Registers = []
with open("C:/Analytics/Lista_de_Ruc.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        Registers.append(row[0])  # Suponiendo que el dato está en la primera columna
Registers

['1790300404001',
 '1391738986001',
 '1391880081001',
 "0891742622001'",
 '1391932105001',
 '1391797486001',
 '1391898800001',
 '1390012949001',
 '1391877986001',
 '1391932601001',
 '1391794495001',
 '1391932573001',
 '1391800673001',
 "0990604282001'",
 '1391822189001',
 '1391815123001',
 '1391933738001']

In [ ]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.iess.gob.ec/empleador-web/pages/morapatronal/certificadoCumplimientoPublico.jsf")
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
for register in Registers:  
    time.sleep(2)
    input_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/table/tbody/tr[2]/td/input")))
    # Ingresar el número en el campo de texto
    input_element.send_keys(register)
    time.sleep(3)
    #Click en boton consultar
    elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/div[5]/input")))
    elemento_xpath.click()
    time.sleep(3)
    
    hora_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
driver.quit()

In [ ]:

#rutas de archivos
downloads_folder = os.path.expanduser("C:/Users/eeras/Downloads")  # Ajusta TU_USUARIO
destination_folder = "C:/Analytics/Iess/pdf"
excel_path = "C:/Analytics/Iess/obligaciones_patronales.xlsx"

pattern = re.compile(r"certificado_empresa_ruc(?: \(\d+\))?\.pdf$")
#mover archivos de descargas a la carpeta analytics
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for file in os.listdir(downloads_folder):
    if file.endswith(".pdf") and pattern.match(file):  
        src_path = os.path.join(downloads_folder, file)
        dest_path = os.path.join(destination_folder, file)
        shutil.move(src_path, dest_path)
        print(f"Movido: {file}")

#Extraer información de PDF
data = []
for file in os.listdir(destination_folder):
    if pattern.match(file):  
        pdf_path = os.path.join(destination_folder, file)
        
        with open(pdf_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"

        #Extraer RUC, Razón Social y Obligaciones Patronales
        ruc_match = re.search(r'RUC Nro. ?\s*(\d{13})', text)
        razon_social_match = re.search(r'representante legal de la empresa\s*([A-Za-z0-9 &]+(?:\s+[A-Za-z0-9 &]+)*)', text)
        obligaciones_match = re.search(r'(NO registra|SI registra)(?:\s*|\n*)', text, re.IGNORECASE)

        ruc = ruc_match.group(1) if ruc_match else "No encontrado"

        #Razon social
        if razon_social_match:
            razon_social = razon_social_match.group(1).strip()
        else:
            razon_social = "No encontrado"

        #obligaciones
        if obligaciones_match:
            obligaciones = obligaciones_match.group(1).strip()
        else:
            obligaciones = "No encontrado"
            
        

        data.append([ruc, razon_social, obligaciones])

#Guardar en un archivo Excel
df = pd.DataFrame(data, columns=["RUC", "Razón Social", "Obligaciones Patronales"])
df.to_excel(excel_path, index=False)

print("Proceso completado. Información guardada en Excel.")




Movido: certificado_empresa_ruc (2).pdf
Proceso completado. Información guardada en Excel.


In [ ]:
import PyPDF2
path= "C:\Analytics\Iess\pdf\certificado_empresa_ruc (2).pdf"

pdf= open(path , "rb")
reader= PyPDF2.PdfReader(pdf)
page=reader._get_page(0)
print(page.extract_text())

CERTIFICADO DE CUMPLIMIENTO DE OBLIGACIONES PATRONALES
 
El Instituto Ecuatoriano de Seguridad Social (IESS) certifica que, revisados los
archivos del Sistema de Historia Laboral, el señor(a) GONZALEZ ARTIGAS
POLANCO JAIME RAMON, representante legal de la empresa EXTRACTORA
AGRICOLA RIO MANSO EXA S.A. con RUC Nro. 1790300404001 y dirección
1. KM 5 1/2 VIA MANTA - MONTECRISTI EN LA FABRIL. 2. 1. 1., NO registra
obligaciones patronales en mora; información verificada a la fecha de emisión
del presente certificado.
El IESS se reserva el derecho de verificar la información y las obligaciones
pendientes que no se encontraren registradas o no hayan sido determinadas,
sin perjuicio de aplicar las sanciones correspondientes;  esta certificación no
implica condonación o renuncia del  derecho del IESS, al ejercicio de las
acciones legales a que hubiere lugar para su cobro.
El contenido de este certificado puede ser validado ingresando al portal web del
IESS en el menú Empleador – Certificado de 